In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType
from pyspark.sql.functions import col, to_date, dayofmonth, dayofweek, month, when, lit, isnull, count
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler, Imputer
from pyspark.ml.classification import RandomForestClassifier, LogisticRegression, GBTClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from datetime import datetime
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Initialize Spark Session optimized for local Jupyter execution
spark = SparkSession.builder \
    .appName("FraudDetection") \
    .master("local[*]") \
    .config("spark.sql.debug.maxToStringFields", "100") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()

print("Spark session created successfully!")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/19 20:14:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created successfully!


In [2]:
# Define schema
schema = StructType([
    StructField("Transaction ID", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Transaction Amount", FloatType(), True),
    StructField("Transaction Date", StringType(), True),
    StructField("Payment Method", StringType(), True),
    StructField("Product Category", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Customer Age", IntegerType(), True),
    StructField("Customer Location", StringType(), True),
    StructField("Device Used", StringType(), True),
    StructField("IP Address", StringType(), True),
    StructField("Shipping Address", StringType(), True),
    StructField("Billing Address", StringType(), True),
    StructField("Is Fraudulent", IntegerType(), True),
    StructField("Account Age Days", IntegerType(), True),
    StructField("Transaction Hour", IntegerType(), True)
])

# Load data with error handling
try:
    # For local Jupyter environment, use local file path - adjust this to your file location
    # If you're using actual HDFS, keep the HDFS path
    try:
        # First try local file
        file_path = "transaction_data.csv"  # Change this to your local file path
        df = spark.read.option("header", "true") \
                       .option("multiLine", "true") \
                       .schema(schema) \
                       .csv(file_path)
    except:
        # If local fails, try HDFS path
        today = datetime.today()
        hdfs_path = f"hdfs://namenode:9000/user/root/transactions/YYYY={today.year}/MM={today.month:02d}/DD={today.day:02d}/transaction_data.csv"
        df = spark.read.option("header", "true") \
                       .option("multiLine", "true") \
                       .schema(schema) \
                       .csv(hdfs_path)
    
    print("Data loaded successfully. Row count:", df.count())
except Exception as e:
    print(f"Error loading data: {str(e)}")
    print("Please update the file path to point to your transaction data.")

Data loaded successfully. Row count: 23634


In [3]:
# Data Quality Check
print("\nData Quality Check:")
print("Null values per column:")
null_counts = df.select([count(when(isnull(c), c)).alias(c) for c in df.columns])
null_counts.show(vertical=True)

# Show sample data
print("\nSample data:")
df.toPandas()



Data Quality Check:
Null values per column:
-RECORD 0-----------------
 Transaction ID     | 0   
 Customer ID        | 0   
 Transaction Amount | 0   
 Transaction Date   | 0   
 Payment Method     | 0   
 Product Category   | 0   
 Quantity           | 0   
 Customer Age       | 0   
 Customer Location  | 0   
 Device Used        | 0   
 IP Address         | 0   
 Shipping Address   | 0   
 Billing Address    | 0   
 Is Fraudulent      | 0   
 Account Age Days   | 0   
 Transaction Hour   | 0   


Sample data:


,Transaction ID,Customer ID,Transaction Amount,Transaction Date,Payment Method,Product Category,Quantity,Customer Age,Customer Location,Device Used,IP Address,Shipping Address,Billing Address,Is Fraudulent,Account Age Days,Transaction Hour
0,c12e07a0-8a06-4c0d-b5cc-04f3af688570,8ca9f102-02a4-4207-ab63-484e83a1bdf0,42.320000,2024-03-24 23:42:43,PayPal,electronics,1,40,East Jameshaven,desktop,110.87.246.85,5399 Rachel Stravenue Suite 718\nNorth Blakebu...,5399 Rachel Stravenue Suite 718\nNorth Blakebu...,0,282,23
1,7d187603-7961-4fce-9827-9698e2b6a201,4d158416-caae-4b09-bd5b-15235deb9129,301.339996,2024-01-22 00:53:31,credit card,electronics,3,35,Kingstad,tablet,14.73.104.153,"5230 Stephanie Forge\nCollinsbury, PR 81853","5230 Stephanie Forge\nCollinsbury, PR 81853",0,223,0
2,f2c14f9d-92df-4aaf-8931-ceaf4e63ed72,ccae47b8-75c7-4f5a-aa9e-957deced2137,340.320007,2024-01-22 08:06:03,debit card,toys & games,5,29,North Ryan,desktop,67.58.94.93,"195 Cole Oval\nPort Larry, IA 58422","4772 David Stravenue Apt. 447\nVelasquezside, ...",0,360,8
3,e9949bfa-194d-486b-84da-9565fca9e5ce,b04960c0-aeee-4907-b1cd-4819016adcef,95.769997,2024-01-16 20:34:53,credit card,electronics,5,45,Kaylaville,mobile,202.122.126.216,"7609 Cynthia Square\nWest Brenda, NV 23016","7609 Cynthia Square\nWest Brenda, NV 23016",0,325,20
4,7362837c-7538-434e-8731-0df713f5f26d,de9d6351-b3a7-4bc7-9a55-8f013eb66928,77.449997,2024-01-16 15:47:23,credit card,clothing,5,42,North Edwardborough,desktop,96.77.232.76,"2494 Robert Ramp Suite 313\nRobinsonport, AS 5...","2494 Robert Ramp Suite 313\nRobinsonport, AS 5...",0,116,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23629,d8b7171f-bdd9-479c-b98b-396c621aebfe,98a3d94c-dc9a-4525-b273-e6ffe54cc5a4,53.730000,2024-01-26 16:25:05,PayPal,toys & games,5,32,Rebeccafurt,tablet,173.97.197.128,USNV Clayton\nFPO AE 82639,USNV Clayton\nFPO AE 82639,0,363,16
23630,0fd12cf3-c641-4499-8de1-15dc4555cb0c,b3429f52-8b27-46b5-914c-4accd989edb4,47.419998,2024-02-25 17:03:26,credit card,clothing,4,41,Nataliefort,desktop,133.222.22.48,"9288 Patricia Cape Apt. 527\nMelissaton, IL 38543","9288 Patricia Cape Apt. 527\nMelissaton, IL 38543",0,296,17
23631,649680d3-a684-44cb-95bf-9b454c3aa86d,066e25c9-4420-4224-bc3f-1a462708090e,1045.229980,2024-03-28 23:46:47,bank transfer,health & beauty,1,9,East Shannonville,tablet,206.133.237.168,3015 Elizabeth Summit Suite 819\nEast Joelfort...,3015 Elizabeth Summit Suite 819\nEast Joelfort...,0,329,23
23632,c10dbb08-28fc-4ec1-9850-d4e98d2b9640,cde96e9c-f562-4b8c-8fa8-f356f474232b,34.250000,2024-02-09 11:29:18,debit card,home & garden,2,39,Lake Nicole,mobile,16.204.137.130,"531 Brittany Pike\nNew Stacy, OR 87952","531 Brittany Pike\nNew Stacy, OR 87952",0,347,11


# Nettoyage & enrichissement des données

In [4]:
from pyspark.sql.functions import col, split, to_timestamp, hour, dayofweek, month, year

# Convert Transaction Date to timestamp
df = df.withColumn("Transaction_Timestamp", to_timestamp(col("Transaction Date")))

# Extract time dimensions
df = df.withColumn("Transaction_DayOfWeek", dayofweek("Transaction_Timestamp")) \
       .withColumn("Transaction_Month", month("Transaction_Timestamp")) \
       .withColumn("Transaction_Year", year("Transaction_Timestamp"))

# Optional: simplify age into age group
def age_group(age):
    if age < 20:
        return "Under 20"
    elif age < 30:
        return "20-29"
    elif age < 40:
        return "30-39"
    elif age < 50:
        return "40-49"
    else:
        return "50+"

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

age_group_udf = udf(age_group, StringType())
df = df.withColumn("Age_Group", age_group_udf(col("Customer Age")))


# Création des tables de dimensions

In [5]:
from pyspark.sql.functions import monotonically_increasing_id
# Dim_Customer
dim_customer = df.select(
    col("Customer ID").alias("Customer_ID"),
    col("Customer Age"),
    col("Customer Location"),
    col("Account Age Days").alias("Account_Age_Days"),
    col("Age_Group")
).dropDuplicates()

# Dim_PaymentMethod
dim_payment = df.select(
    col("Payment Method").alias("Payment_Method")
).dropDuplicates() \
 .withColumn("Payment_Method_ID", monotonically_increasing_id())

# Dim_ProductCategory
dim_product = df.select(
    col("Product Category").alias("Product_Category")
).dropDuplicates() \
 .withColumn("Product_Category_ID", monotonically_increasing_id())

# Dim_Device
dim_device = df.select(
    col("Device Used").alias("Device_Type")
).dropDuplicates() \
 .withColumn("Device_ID", monotonically_increasing_id())

# Dim_Time
dim_time = df.select(
    col("Transaction_Timestamp").alias("Time_ID"),
    col("Transaction Hour").alias("Transaction_Hour"),
    col("Transaction_DayOfWeek"),
    col("Transaction_Month"),
    col("Transaction_Year")
).dropDuplicates()




# Table des faits : Fact_Transactions

In [6]:
# Join payment method
fact_df = df.join(dim_payment, df["Payment Method"] == dim_payment["Payment_Method"], "left") \
            .join(dim_product, df["Product Category"] == dim_product["Product_Category"], "left") \
            .join(dim_device, df["Device Used"] == dim_device["Device_Type"], "left")

In [7]:
fact_transactions = fact_df.select(
    col("Transaction ID").alias("Transaction_ID"),
    col("Customer ID").alias("Customer_ID"),
    col("Transaction Amount").alias("Transaction_Amount"),
    col("Quantity"),
    col("Is Fraudulent").alias("Is_Fraudulent"),
    col("Transaction_Timestamp"),
    col("Payment_Method_ID"),
    col("Product_Category_ID"),
    col("Device_ID")
)


In [8]:
print("Dimensions:")
print("Customers:", dim_customer.count())
print("Payment Methods:", dim_payment.count())
print("Product Categories:", dim_product.count())
print("Devices:", dim_device.count())
print("Time:", dim_time.count())

print("\nSample from Fact Transactions:")
fact_transactions.show(5, truncate=False)


Dimensions:


Customers: 23634
Payment Methods: 4
Product Categories: 5
Devices: 3
Time: 23609

Sample from Fact Transactions:
+------------------------------------+------------------------------------+------------------+--------+-------------+---------------------+-----------------+-------------------+---------+
|Transaction_ID                      |Customer_ID                         |Transaction_Amount|Quantity|Is_Fraudulent|Transaction_Timestamp|Payment_Method_ID|Product_Category_ID|Device_ID|
+------------------------------------+------------------------------------+------------------+--------+-------------+---------------------+-----------------+-------------------+---------+
|c12e07a0-8a06-4c0d-b5cc-04f3af688570|8ca9f102-02a4-4207-ab63-484e83a1bdf0|42.32             |1       |0            |2024-03-24 23:42:43  |2                |2                  |0        |
|7d187603-7961-4fce-9827-9698e2b6a201|4d158416-caae-4b09-bd5b-15235deb9129|301.34            |3       |0            |2024-01-22 00:53:3

In [9]:
# Convert dimensions to Pandas DataFrames
dim_customers_pd = dim_customer.toPandas()
dim_payment_methods_pd = dim_payment.toPandas()
dim_product_categories_pd = dim_product.toPandas()
dim_devices_pd = dim_device.toPandas()
dim_time_pd = dim_time.toPandas()

# Convert fact table to Pandas DataFrame
fact_transactions_pd = fact_transactions.toPandas()

# Show the tables 

In [10]:
print("Dim_Customers DataFrame:")
dim_customers_pd.head()

Dim_Customers DataFrame:


,Customer_ID,Customer Age,Customer Location,Account_Age_Days,Age_Group
0,bfc4dcdf-37af-43dc-95ae-811607411c86,32,Michaelchester,232,30-39
1,af735f51-f712-4ca1-a0fd-686768c3bc97,51,Port Janet,189,50+
2,cdd9f0d5-0f30-44bd-8e66-addba05d317d,33,Jonesshire,270,30-39
3,6b22e12c-1fcb-49a6-b7bb-5e713e0e4b39,23,North Christopher,235,20-29
4,60845d99-3c65-4247-86f3-cad3d35e9700,30,Timothyport,285,30-39


In [11]:
print("\nDim_Payment_Methods DataFrame:")
dim_payment_methods_pd.head()


Dim_Payment_Methods DataFrame:


,Payment_Method,Payment_Method_ID
0,debit card,0
1,bank transfer,1
2,PayPal,2
3,credit card,3


In [12]:
print("\nDim_Product_Categories DataFrame:")
dim_product_categories_pd.head(7)


Dim_Product_Categories DataFrame:


,Product_Category,Product_Category_ID
0,health & beauty,0
1,toys & games,1
2,electronics,2
3,clothing,3
4,home & garden,4


In [13]:
print("\nDim_Devices DataFrame:")
dim_devices_pd.head()


Dim_Devices DataFrame:


,Device_Type,Device_ID
0,desktop,0
1,mobile,1
2,tablet,2


In [14]:
print("\nDim_Time DataFrame:")
dim_time_pd.head()


Dim_Time DataFrame:


,Time_ID,Transaction_Hour,Transaction_DayOfWeek,Transaction_Month,Transaction_Year
0,2024-03-08 17:49:54,17,6,3,2024
1,2024-02-26 01:29:55,1,2,2,2024
2,2024-01-10 07:13:00,7,4,1,2024
3,2024-01-02 16:10:28,16,3,1,2024
4,2024-03-03 21:47:05,21,1,3,2024


In [15]:
print("\nFact_Transactions DataFrame:")
fact_transactions_pd.head()


Fact_Transactions DataFrame:


,Transaction_ID,Customer_ID,Transaction_Amount,Quantity,Is_Fraudulent,Transaction_Timestamp,Payment_Method_ID,Product_Category_ID,Device_ID
0,c12e07a0-8a06-4c0d-b5cc-04f3af688570,8ca9f102-02a4-4207-ab63-484e83a1bdf0,42.320000,1,0,2024-03-24 23:42:43,2,2,0
1,7d187603-7961-4fce-9827-9698e2b6a201,4d158416-caae-4b09-bd5b-15235deb9129,301.339996,3,0,2024-01-22 00:53:31,3,2,2
2,f2c14f9d-92df-4aaf-8931-ceaf4e63ed72,ccae47b8-75c7-4f5a-aa9e-957deced2137,340.320007,5,0,2024-01-22 08:06:03,0,1,0
3,e9949bfa-194d-486b-84da-9565fca9e5ce,b04960c0-aeee-4907-b1cd-4819016adcef,95.769997,5,0,2024-01-16 20:34:53,3,2,1
4,7362837c-7538-434e-8731-0df713f5f26d,de9d6351-b3a7-4bc7-9a55-8f013eb66928,77.449997,5,0,2024-01-16 15:47:23,3,3,0


In [16]:
from datetime import datetime

today = datetime.today()
year = today.year
month = f"{today.month:02d}"
day = f"{today.day:02d}"


In [17]:
base_path = "hdfs://namenode:9000/user/root/datalake"
date_path = f"YYYY={year}/MM={month}/DD={day}"

In [18]:
# Full paths with partition folders
fact_path = f"{base_path}/fact_transactions/{date_path}"
customers_path = f"{base_path}/dim_customers/{date_path}"
payments_path = f"{base_path}/dim_payment_methods/{date_path}"
products_path = f"{base_path}/dim_product_categories/{date_path}"
devices_path = f"{base_path}/dim_devices/{date_path}"
time_path = f"{base_path}/dim_time/{date_path}"

# Save each DataFrame to HDFS
fact_transactions.write.mode("overwrite") \
    .option("header", "true") \
    .csv(fact_path)

dim_customer.write.mode("overwrite") \
    .option("header", "true") \
    .csv(customers_path)

dim_payment.write.mode("overwrite") \
    .option("header", "true") \
    .csv(payments_path)

dim_product.write.mode("overwrite") \
    .option("header", "true") \
    .csv(products_path)

dim_device.write.mode("overwrite") \
    .option("header", "true") \
    .csv(devices_path)

dim_time.write.mode("overwrite") \
    .option("header", "true") \
    .csv(time_path)


🎯 BIG QUESTIONS & VISUALIZATIONS
1. What are the most common fraud patterns?
💡 Why it matters: Helps detect repeat behavior and improve fraud detection models.

📊 Visualization:

Bar chart: Top 10 Product Categories in Fraudulent Transactions

Heatmap: Fraud Count by Hour of Day vs Device Type

2. When do most transactions happen?
💡 Why it matters: Optimizes system resources and security monitoring in peak hours.

📊 Visualization:

Histogram or Line chart: Number of Transactions per Hour (0-23)

Line chart: Daily Transaction Count over Time

3. What is the geographical spread of fraudulent transactions?
💡 Why it matters: Identifies locations or regions at higher risk.

📊 Visualization:

Map (Geo-plot): Fraud Count by Customer Location

4. Which payment methods are more associated with fraud?
💡 Why it matters: Can guide business decisions around enabling/disabling payment options.

📊 Visualization:

Pie chart or Bar chart: Fraud Count by Payment Method

5. Are older accounts more likely to be safe?
💡 Why it matters: Answers whether new accounts are riskier.

📊 Visualization:

Box plot or Scatter plot: Account Age vs Fraud Probability

Could use a regression line for trend analysis.

6. Which devices are riskier for transactions?
💡 Why it matters: Detect fraud from mobile vs desktop.

📊 Visualization:

Stacked bar chart: Device Type vs Fraud Count

7. High-value transactions and fraud – any correlation?
💡 Why it matters: Bigger amounts are tempting targets.

📊 Visualization:

Scatter plot: Transaction Amount vs Fraud Label (0/1)

Color points by Product Category

⚡ Bonus Insight: Anomaly Detection Visual
Use a time-series line graph to mark spikes (anomalies) in transaction amount or fraud percentage.

💻 Want the Python or PySpark code to generate some of these visuals using matplotlib, seaborn, or pandas?
Or would you like help setting them up in Power BI / Tableau using your CSVs from HDFS?

Let me know your preferred tool and I’ll tailor it for you.

In [19]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Load the CSV
# df = pd.read_csv("transaction_data.csv")

# # Optional: Adjust plot style
# sns.set(style="whitegrid")
# plt.figure(figsize=(12, 6))

# # --- 1. Top 10 Product Categories in Fraudulent Transactions ---
# plt.figure(figsize=(10, 6))
# fraud_by_category = df[df["Is Fraudulent"] == 1]["Product Category"].value_counts().head(10)
# sns.barplot(x=fraud_by_category.values, y=fraud_by_category.index, palette="Reds_r")
# plt.title("Top 10 Product Categories in Fraudulent Transactions")
# plt.xlabel("Number of Fraudulent Transactions")
# plt.ylabel("Product Category")
# plt.tight_layout()
# plt.show()

# # --- 2. Transactions by Hour ---
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df, x="Transaction Hour", palette="viridis")
# plt.title("Number of Transactions by Hour")
# plt.xlabel("Hour of Day")
# plt.ylabel("Transaction Count")
# plt.tight_layout()
# plt.show()

# # --- 3. Fraud Count by Customer Location ---
# plt.figure(figsize=(10, 6))
# fraud_by_location = df[df["Is Fraudulent"] == 1]["Customer Location"].value_counts().head(10)
# sns.barplot(x=fraud_by_location.values, y=fraud_by_location.index, palette="coolwarm")
# plt.title("Top Locations with Most Fraudulent Transactions")
# plt.xlabel("Number of Fraud Cases")
# plt.ylabel("Customer Location")
# plt.tight_layout()
# plt.show()

# # --- 4. Fraud Count by Payment Method ---
# plt.figure(figsize=(8, 6))
# fraud_by_payment = df[df["Is Fraudulent"] == 1]["Payment Method"].value_counts()
# sns.barplot(x=fraud_by_payment.index, y=fraud_by_payment.values, palette="Blues")
# plt.title("Fraud Count by Payment Method")
# plt.xlabel("Payment Method")
# plt.ylabel("Fraud Count")
# plt.tight_layout()
# plt.show()

# # --- 5. Account Age vs Fraud (Boxplot) ---
# plt.figure(figsize=(10, 6))
# sns.boxplot(x="Is Fraudulent", y="Account Age Days", data=df, palette="Set2")
# plt.title("Account Age vs Fraud")
# plt.xlabel("Is Fraudulent")
# plt.ylabel("Account Age (Days)")
# plt.xticks([0, 1], ["No", "Yes"])
# plt.tight_layout()
# plt.show()

# # --- 6. Device Type vs Fraud Count ---
# plt.figure(figsize=(10, 6))
# sns.countplot(data=df, x="Device Used", hue="Is Fraudulent", palette="muted")
# plt.title("Device Type vs Fraudulent vs Non-Fraudulent Transactions")
# plt.xlabel("Device Used")
# plt.ylabel("Transaction Count")
# plt.legend(title="Is Fraudulent", labels=["No", "Yes"])
# plt.tight_layout()
# plt.show()

# # --- 7. Transaction Amount vs Fraud (Scatter Plot) ---
# plt.figure(figsize=(10, 6))
# sns.scatterplot(data=df, x="Transaction Amount", y="Account Age Days", hue="Is Fraudulent", alpha=0.6)
# plt.title("Transaction Amount vs Account Age (Colored by Fraud)")
# plt.xlabel("Transaction Amount")
# plt.ylabel("Account Age (Days)")
# plt.legend(title="Is Fraudulent")
# plt.tight_layout()
# plt.show()
